Trabalho final
Cristiano Henrrique Araujo Miranda
202008076481

In [1]:
#Instalando o Java.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Baixando o Spark.
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#Descompactando arquivo do Spark.
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#Instalando a lib Findspark para localizar o Spark no sistema e imoporta-lo como uma blibioteca regular.
!pip install -q findspark
#Insatalado PySpark para trablharmos com o Spark em Python 
!pip install -q pyspark
#Instalado o Plotly
!pip install -q plotly

The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.1.2-bin-hadoop3.2.tgz'
'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
#Criando variáveis de ambiente 
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop3.2'
#Importando a lib findspark
import findspark
findspark.init()
#Importando pandas
import pandas as pd
#Importando Plotly
import plotly.express as px
#Criando Spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('ses_TrabBgDaPy').getOrCreate()

In [3]:
#Imporando arquivo
from google.colab import files
arquivo = files.upload()

Saving 2022_janeiro_arrecadacao_mensal.csv to 2022_janeiro_arrecadacao_mensal.csv


In [4]:
#Especificões dos tipos de dados de entrada  
from pyspark.sql.types import *
schema_df = StructType([
                    StructField("receita",StringType(),False),
                    StructField("nome_receita",StringType(),False),
                    StructField("escrituracao",StringType(),False),
                    StructField("Janeiro",DecimalType(20,2),True),
                    StructField("Fevereiro",DecimalType(20,2),True),
                    StructField("Março",DecimalType(20,2),True),
                    StructField("Abril",DecimalType(20,2),True),
                    StructField("Maio",DecimalType(20,2),True),
                    StructField("Junho",DecimalType(20,2),True),
                    StructField("Julho",DecimalType(20,2),True),
                    StructField("Agosto",DecimalType(20,2),True),
                    StructField("Setembro",DecimalType(20,2),True),
                    StructField("Outubro",DecimalType(20,2),True),
                    StructField("Novembro",DecimalType(20,2),True),
                    StructField("Dezembro",DecimalType(20,2),True)])

#criando dataframe
df = spark.read.csv('2022_janeiro_arrecadacao_mensal.csv', 
                    sep=';',
                    encoding='latin1', 
                    locale='pt-BR', 
                    header=True, 
                    schema=schema_df, 
                    mode='FAILFAST',  
                    nullValue='-', 
                    ignoreLeadingWhiteSpace=True,
                    ignoreTrailingWhiteSpace=True)

df.printSchema()

root
 |-- receita: string (nullable = true)
 |-- nome_receita: string (nullable = true)
 |-- escrituracao: string (nullable = true)
 |-- Janeiro: decimal(20,2) (nullable = true)
 |-- Fevereiro: decimal(20,2) (nullable = true)
 |-- Março: decimal(20,2) (nullable = true)
 |-- Abril: decimal(20,2) (nullable = true)
 |-- Maio: decimal(20,2) (nullable = true)
 |-- Junho: decimal(20,2) (nullable = true)
 |-- Julho: decimal(20,2) (nullable = true)
 |-- Agosto: decimal(20,2) (nullable = true)
 |-- Setembro: decimal(20,2) (nullable = true)
 |-- Outubro: decimal(20,2) (nullable = true)
 |-- Novembro: decimal(20,2) (nullable = true)
 |-- Dezembro: decimal(20,2) (nullable = true)



In [5]:
#Mostrando Datafreme para validar se não há inconsitencia nos dados 
try:
  df.show()
except:
  print('Erro no DataFreme') 
else:
  df_VeriRegNullR = df.filter(df.receita.isNull())
  df_VeriRegNullN = df.filter(df.nome_receita.isNull())
  df_VeriRegNullE = df.filter(df.escrituracao.isNull())
  df_VeriRegRepet = df.groupBy('receita').count()
  df_VeriRegRepet = df_VeriRegRepet.filter(df_VeriRegRepet['count'] > 1)
#Verificando se há registros vazio
  if(df.rdd.isEmpty()):
    print('DataFreme está vasio')
#Verificando se há null
  elif (df_VeriRegNullR.count() > 0 or df_VeriRegNullN.count() > 0 or df_VeriRegNullE.count() > 0):
    print(f'Há {df_VeriRegNullR.count()+df_VeriRegNullN.count()+df_VeriRegNullE.count()} registros com receita NULL')
    print('receita com NULL')
    df_VeriRegNullR.show()
    print('nome_receita com NULL')
    df_VeriRegNullN.show()
    print('escrituracao com NULL')
    df_VeriRegNullE.show()
#Verificando se há regiastro repitido       
  elif(df_VeriRegRepet.count() > 0 ):
    print(f'Há {df_VeriRegRepet.count()} registros repetidos com mesma receita')
    df_VeriRegRepet.show()
  else:
    print('DataFrame está ok')

+--------------+--------------------+------------+-------------+---------+-----+-----+----+-----+-----+------+--------+-------+--------+--------+
|       receita|        nome_receita|escrituracao|      Janeiro|Fevereiro|Março|Abril|Maio|Junho|Julho|Agosto|Setembro|Outubro|Novembro|Dezembro|
+--------------+--------------------+------------+-------------+---------+-----+-----+----+-----+-----+------+--------+-------+--------+--------+
|1.0.0.0.00.0.0|  Receitas Correntes|           N|1887415297.18|     null| null| null|null| null| null|  null|    null|   null|    null|    null|
|1.1.0.0.00.0.0|Impostos, Taxas e...|           N|1238343889.69|     null| null| null|null| null| null|  null|    null|   null|    null|    null|
|1.1.1.0.00.0.0|            Impostos|           N|1065012499.19|     null| null| null|null| null| null|  null|    null|   null|    null|    null|
|1.1.1.2.00.0.0|Impostos sobre o ...|           N| 836741987.40|     null| null| null|null| null| null|  null|    null|   nu

In [6]:
#a)Quantidade de registros 
quant_Reg = df.count()
print(f'Exite um total de {quant_Reg} registro no DataFrame.\n')


Exite um total de 341 registro no DataFrame.



In [7]:
#b)Colunas constantes
print(f'No DataFrame existem as seguintes colunas:')
colum_df = df.columns
for col in colum_df:
  print(col)

No DataFrame existem as seguintes colunas:
receita
nome_receita
escrituracao
Janeiro
Fevereiro
Março
Abril
Maio
Junho
Julho
Agosto
Setembro
Outubro
Novembro
Dezembro


In [8]:
#c)Verifica campos com registros faltantes
df_VeriRegNullR = df.filter(df.receita.isNull())
df_VeriRegNullN = df.filter(df.nome_receita.isNull())
df_VeriRegNullE = df.filter(df.escrituracao.isNull())
df_VeriRegNullJ = df.filter(df.Janeiro.isNull())
if(df.rdd.isEmpty()):
  print('DataFreme está vasio')
elif(df_VeriRegNullR.count() > 0 or df_VeriRegNullN.count() > 0 or df_VeriRegNullE.count() > 0 or df_VeriRegNullJ.count() > 0):
  print(f'Há {df_VeriRegNullR.count()+df_VeriRegNullN.count()+df_VeriRegNullE.count()} registros com receita NULL')
  print('receita com NULL')
  df_VeriRegNullR.show()
  print('nome_receita com NULL')
  df_VeriRegNullN.show()
  print('escrituracao com NULL')
  df_VeriRegNullE.show()
  print('Janeiro com NULL')
  df_VeriRegNullE.show()
else:
  print('Não ha campos faltantes')


Não ha campos faltantes


In [9]:
#d)Valores recebidos por nome da receita no mês selecionado.
df_PorNomeReceita = df.groupBy('nome_receita').agg({'Janeiro': 'sum'})
df_PorNomeReceita.orderBy('nome_receita').show()



+--------------------+------------+
|        nome_receita|sum(Janeiro)|
+--------------------+------------+
|   Alienação de Bens|  4305772.65|
|Alienação de Bens...|   319882.56|
|Alienação de Bens...|   106627.52|
|Alienação de Bens...|  4199145.13|
|Alienação de Títu...|  4199145.13|
|Alienação de Títu...|  4199145.13|
|Alienação de Títu...|  4199145.13|
|Aluguéis e Arrend...|   109137.05|
|Aluguéis e Arrend...|      567.65|
|Aluguéis e Arrend...|       42.14|
|Aluguéis e Arrend...|       11.31|
|Aluguéis e Arrend...|   108515.95|
|Aluguéis, Arrenda...|   114891.05|
|Compensações Fina...|   888989.92|
|Compensações Fina...|   888989.92|
|Concessão, Permis...|    18643.80|
|Concessão, Permis...|    18643.80|
|Contribuição Oriu...|    22358.77|
|Contribuição Oriu...|    22358.77|
|Contribuição Oriu...|   673797.84|
+--------------------+------------+
only showing top 20 rows



In [10]:
#e)valor recebido por Receitas Correntes no mês selecionado.
df_PorReceita = df.groupBy('receita').agg({'Janeiro': 'sum'})
df_PorReceita.orderBy('receita').show(truncate=False)

+--------------+-------------+
|receita       |sum(Janeiro) |
+--------------+-------------+
|1.0.0.0.00.0.0|1887415297.18|
|1.1.0.0.00.0.0|1238343889.69|
|1.1.1.0.00.0.0|1065012499.19|
|1.1.1.2.00.0.0|836741987.40 |
|1.1.1.2.50.0.0|798235589.00 |
|1.1.1.2.50.0.1|783402766.76 |
|1.1.1.2.50.0.3|10052790.72  |
|1.1.1.2.50.0.5|62522.96     |
|1.1.1.2.50.0.6|130402.15    |
|1.1.1.2.50.0.7|2206986.96   |
|1.1.1.2.50.0.8|2380119.45   |
|1.1.1.2.53.0.0|38506398.40  |
|1.1.1.2.53.0.1|38130017.49  |
|1.1.1.2.53.0.3|311607.58    |
|1.1.1.2.53.0.5|3038.65      |
|1.1.1.2.53.0.6|9706.42      |
|1.1.1.2.53.0.7|16635.83     |
|1.1.1.2.53.0.8|35392.43     |
|1.1.1.3.00.0.0|40349204.13  |
|1.1.1.3.03.0.0|40349204.13  |
+--------------+-------------+
only showing top 20 rows



In [11]:
#f)Média do valor recebido.
df_MediaJaneiro = df.agg({'Janeiro': 'avg'})
df_MediaJaneiro.show()

+---------------+
|   avg(Janeiro)|
+---------------+
|35020354.771290|
+---------------+



In [12]:
#g)Valor total recebido pelo município no mês selecionado.
df_MediaJaneiro = df.agg({'Janeiro': 'sum'})
df_MediaJaneiro.show()

+--------------+
|  sum(Janeiro)|
+--------------+
|11941940977.01|
+--------------+



In [13]:
#passando datafrem spark para pandas
import numpy as np
df_pd = df.toPandas()
df_pd = df_pd.replace(np.nan, 0, regex= True)
df_pd["Janeiro"] = df_pd["Janeiro"].astype(float)
print(df_pd["Janeiro"])

df_ppp = df_PorNomeReceita.toPandas()
df_ppp = df_ppp.replace(np.nan, 0, regex= True)
df_ppp["sum(Janeiro)"] = df_ppp["sum(Janeiro)"].astype(float)


graf1 = df_ppp.nlargest(15, "sum(Janeiro)")
graf1 = px.bar(graf1,x="nome_receita", y="sum(Janeiro)")
graf1.show()


graf2 = px.pie(df_pd, names="escrituracao", values="Janeiro")
graf2.show() 

graf3 = df_pd.nlargest(15, "Janeiro")
graf3 = px.line(graf3,x="receita", y="Janeiro")
graf3.show()



0      1.887415e+09
1      1.238344e+09
2      1.065012e+09
3      8.367420e+08
4      7.982356e+08
           ...     
336   -2.395888e+07
337   -1.335231e+07
338   -1.335231e+07
339   -2.690271e+05
340   -2.690271e+05
Name: Janeiro, Length: 341, dtype: float64
